In [1]:
!pip install dotenv

In [2]:
# ============================================================
# 0. Настройки среды
# ============================================================
import sys
import os

# Абсолютный путь к корню проекта
project_root = os.path.abspath(os.path.join(".."))  # если notebooks/ внутри корня
if project_root not in sys.path:
    sys.path.insert(0, project_root)

from utils.config_loader import load_config

ENV = "local"
CONFIG = load_config(ENV, project_root=project_root)

print(f"Running in {ENV.upper()} mode")
print(CONFIG)


Running in LOCAL mode
{'DATA_PATH': './data/sample.csv', 'USE_GPU': False, 'EMBEDDING_MODEL': 'all-MiniLM-L6-v2', 'MONGO_URI': 'mongodb://localhost:27017', 'QDRANT_URL': 'http://localhost:6333'}
